Importing my engine.py file

Note: instead of mentioning how each function, which has been imported from engine.p, works, engine.py is well commented and easy to read! Kindly check it out in parallel. 

In [1]:
from engine import *

[nltk_data] Downloading package punkt to /Users/saifdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saifdev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


First, I will load all (approximately) 6000 tsv files into one final dataset.

In [2]:
directory = "/Users/saifdev/Desktop/ADMHW3/files_tsv"
output_file = "/Users/saifdev/Desktop/ADMHW3/compiled.tsv"
dataset = create_store_tsv(directory, output_file)

Now, I will make a copy to protect the original dataset

In [3]:
df = dataset.copy()
df.head(3)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Full time,Front-end DevelopmentatHarbour.Space Universit...,"September, January","€29,900/year",MSc,1 year,Barcelona,Spain,On Campus,www.findamasters.com/masters-degrees/course/ma...
1,IMETE,IHE Delft Institute for Water Education,Graduate School,Full time,IMETEis a joint MSc programme with a duration ...,September,"€18,000.00 total for 2-year programme, to be p...",MSc,2 Years Full Time,Delft,Netherlands,On Campus,www.findamasters.com/masters-degrees/course/im...
2,Clinical Neuropsychology - MSc,University of Bristol,Faculty of Life Sciences,Full time&Part time,Professional programmes in Clinical Neuropsych...,"September, January",Please see the university website for further ...,MSc,"1 year full-time, 2-3 years part-time",Bristol,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/cl...


As we will be working with the description column first I will check the number of missing value. If there are, I will replace them with "NA".

In [4]:
df.description.isnull().sum() # outputs 21
df['description'].fillna("NA", inplace=True)

# 2. Search Engine
## 2.0.0 Preprocessing the text
First, you must pre-process all the information collected for each MSc by:

  * Removing stopwords
  * Removing punctuation
  * Stemming
  * Anything else you think it's needed


Now, as the dataset is loaded, I will apply some preprocessing to all columns. This pre-processing includes, tokenizing, standardazing the text (lower case and removing stop words), and stemming the text of all columns.

In [5]:
desired_columns = [x for x in df.columns if x != "fees"]
for column in desired_columns:
    df[column+"_clean"] = df[column].apply(lambda x: preprocessing(x))

In [6]:
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,...,facultyName_clean,isItFullTime_clean,description_clean,startDate_clean,modality_clean,duration_clean,city_clean,country_clean,administration_clean,url_clean
0,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Full time,Front-end DevelopmentatHarbour.Space Universit...,"September, January","€29,900/year",MSc,1 year,Barcelona,...,"[master, programm]","[full, time]","[front, end, developmentatharbour, space, univ...","[septemb, januari]",[msc],[year],[barcelona],[spain],[campu],"[www, findamast, com, master, degre, cours, ma..."
1,IMETE,IHE Delft Institute for Water Education,Graduate School,Full time,IMETEis a joint MSc programme with a duration ...,September,"€18,000.00 total for 2-year programme, to be p...",MSc,2 Years Full Time,Delft,...,"[graduat, school]","[full, time]","[imetei, joint, msc, programm, durat, month, o...",[septemb],[msc],"[year, full, time]",[delft],[netherland],[campu],"[www, findamast, com, master, degre, cours, imet]"
2,Clinical Neuropsychology - MSc,University of Bristol,Faculty of Life Sciences,Full time&Part time,Professional programmes in Clinical Neuropsych...,"September, January",Please see the university website for further ...,MSc,"1 year full-time, 2-3 years part-time",Bristol,...,"[faculti, life, scienc]","[full, time, part, time]","[profession, programm, clinic, neuropsycholog,...","[septemb, januari]",[msc],"[year, full, time, year, part, time]",[bristol],"[unit, kingdom]",[campu],"[www, findamast, com, master, degre, cours, cl..."
3,Demography and Social Inequality - Double Masters,University of Cologne,"Faculty of Management, Economics and Social Sc...",Full time,The Double Master's Programme in Demography an...,October,At the University of Cologne there are no tuit...,"MSc,Other",2 years full time,Cologne,...,"[faculti, manag, econom, social, scienc]","[full, time]","[doubl, programm, demographi, social, inequ, g...",[octob],[msc],"[year, full, time]",[cologn],[germani],[campu],"[www, findamast, com, master, degre, cours, de..."
4,MSc Finance and Investment Management,University of Liverpool,Liverpool Online Programmes,Part time,A career in finance and investment management ...,"January, May","Fees for the academic year 2022/23MSc: £15,300...","MSc,PGCert,PGDip",2.5 years,Liverpool,...,"[liverpool, onlin, programm]","[part, time]","[career, financ, invest, manag, thrill, reward...","[januari, may]","[msc, pgcert, pgdip]",[year],[liverpool],"[unit, kingdom]",[onlin],"[www, findamast, com, master, degre, cours, ms..."


Now, it is time to do some preprocssing with the fees column. We have to ensure that all the fees are in Euro (decided) and the column is of dtype float. So, let's do it!

First, I will replace all the entries that are prompting to contact the university or visit webpage. Then, I will see what currency symbols have been used through out the dataset so that I can desing my regex accordingly.

In [7]:
if_website_or_contact_replace(df, "fees")
df["fees_clean"] = df["fees_clean"].apply(lambda x: "Not Available" if pd.isnull(x) else x)
symbols_list = extract_symbols(df)
symbols_list

['€', '£', '$']

Now, from all of the text in each row of the "fees" column, I will only keep the max amount and the currency name.

In [8]:
df["fees_clean"] = df["fees_clean"].apply(lambda x: extract_numeric_and_currency(x))
df.loc[:, ["fees","fees_clean"]]

,fees,fees_clean
0,"€29,900/year",29900.0 EUR
1,"€18,000.00 total for 2-year programme, to be p...",18000.0 EUR
2,Please see the university website for further ...,Not Available
3,At the University of Cologne there are no tuit...,300.0 EUR
4,"Fees for the academic year 2022/23MSc: £15,300...",16065.0 GBP
...,...,...
5995,"Full time - £14,100Part time - £7,050",14100.0 GBP
5996,Please see the university website for further ...,Not Available
5997,Please see the university website for further ...,Not Available
5998,NaN,Not Available


Now, I will create two more columns and use these columns to make make a new column FEE_EUR where all the fees are in EUR.

In [11]:
# Apply extraction functions to create new columns
df['numeric_value'] = df['fees_clean'].apply(extract_numeric_value)
df['currency_name'] = df['fees_clean'].apply(extract_currency_name)

For the purpose of converting all other currencies to EUR, I will use v6.exchangerate-api.com as suggested by Chat GPT. I have already signed up and acquired my api key. 

In [12]:
my_api_key = "086fdd533cda265f02aa74ac"
url = f"https://v6.exchangerate-api.com/v6/{my_api_key}/latest/EUR" # to access the url with EUR rates
response = requests.get(url) # to get the latest eur to other currencies rates
data = response.json() # storing the data in the variable data

# Apply the conversion using a lambda function with our pre-defined convert_to_eur function
df['FEE_EUR'] = df.apply(lambda row: convert_to_eur(row['numeric_value'], row['currency_name'], data), axis=1)

Finally, I will delete the extra columns. Namely: "fees_clean", "numeric_value", "currency_name"

In [13]:
df.columns
columns_to_drop = ["fees_clean", "numeric_value", "currency_name"]
df = df.drop(columns=columns_to_drop)

In [14]:
# Check if the data type of FEE_EUR column is float:
df['FEE_EUR'].dtype

dtype('float64')

## 2.1. Conjunctive query

2.1.1 Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

In [15]:
vocabulary = set() # set to remove duplicates and efficiency
# I will only used the description column as stated in the HW
df.description_clean.apply(lambda row: [vocabulary.add(word) for word in row])
vocabulary = list(vocabulary)

The variable vocabulary contains, in a list, all the words used in the description column. The question requires us to make a file named vocabulary that *maps each word to an integer (term_id).*

In [16]:
# maps each word to an integer (term_id)
vocab_with_index = dict()
unique_id = 0
for word in vocabulary:
  vocab_with_index[word] = unique_id
  unique_id+=1
vocab_with_index

with open("/Users/saifdev/Desktop/ADMHW3/vocabulary.json", "w") as file: # storing for reusability
    json.dump(vocab_with_index, file)

In [17]:
vocab_with_index = json.load(open("/Users/saifdev/Desktop/ADMHW3/vocabulary.json", "r")) # loading for use

#### 2.1.1
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}

To create the inverted index, I can use the procedure used in the lab as follows:

In [18]:
Terms = pd.DataFrame(data=vocab_with_index.keys(), columns=['Term']); tqdm.pandas() # from lab
Terms['reverse'] = Terms.Term.progress_apply(lambda item: list(df.loc[df.description_clean.apply(lambda row: item in row)].index))

# word_and_appearances help with the search engine and inverted index. It contains as keys the words and as values a list which 
# represents the occurences of that word in our dataset

word_and_appearances = dict(zip(Terms.Term, Terms.reverse)) 

# The inverted index can also be made as commented below: 
# inverted_index = dict(zip(Terms.index, Terms['reverse'])); however I will implement my own function
Terms.head()

100%|██████████| 9705/9705 [00:44<00:00, 219.97it/s]


,Term,reverse
0,unsustain,"[1551, 3343, 3671]"
1,huge,"[10, 74, 217, 577, 595, 765, 976, 1023, 1167, ..."
2,beneath,[2535]
3,redesign,"[2108, 2354]"
4,b,"[127, 148, 453, 1890, 2871, 3581, 3778, 4916, ..."


OR, I can use the function that I created to make an inverted list. Here's how:

In [19]:
inverted_index = inverted_index(vocab_with_index, df)

I will now store this inverted index locally

In [20]:
with open("/Users/saifdev/Desktop/ADMHW3/inverted_index_1.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("/Users/saifdev/Desktop/ADMHW3/inverted_index_1.json", "r")) # loading for use

2.1.2) Execute the query
Given a query input by the user, for example:

advanced knowledge

The Search Engine is supposed to return a list of documents.

For this task, I will first take the input from the user and then normalize the query using the same technique I used to preprocess the description column.

In [21]:
user_input = input("Add your Input")
normalized_query = normalize_query(user_input)

Instead of using the inverted index which has the term_id and a list of where it appears in the document, I will use a related inverted index, which I created above with name word_and_appearances which contains words as keys and list of places they occur in the document as values.

In [22]:
new_dataset = search_engine(normalized_query, df, word_and_appearances).head()
new_dataset

,courseName,universityName,description,url
0,Clinical Neuropsychology - MSc,University of Bristol,Professional programmes in Clinical Neuropsych...,www.findamasters.com/masters-degrees/course/cl...
1,MSc In People Analytics,University of Hull,Start date: January 2024Play an essential role...,www.findamasters.com/masters-degrees/course/ms...
2,Master of Science in Professional Nursing,Atlantic Technological University,The programme provides the nurse with a broad-...,www.findamasters.com/masters-degrees/course/ma...
3,"Data, Inequality and Society MSc, PgDip (ICL),...",University of Edinburgh,Programme descriptionAn interdisciplinary degr...,www.findamasters.com/masters-degrees/course/da...
4,Digital Design and Branding MSc,Brunel University London,Ask BrunelOur Digital Design and Branding MSc ...,www.findamasters.com/masters-degrees/course/di...


## 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

Find all the documents that contain all the words in the query. Sort them by their similarity with the query.

Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description.

##### 2.2.1) Make an Inverted index

Below, I will make an inverted index which contains as keys the unique term_id of the word, and as values, its corresponding occurences and tf-idf values in the dataset's row.

In [23]:
# Our inverted_index with tf-idf value as well.
inverted_index_tf_idf = inverted_index_tfidf(word_and_appearances, df)

In [24]:
with open("/Users/saifdev/Desktop/ADMHW3/inverted_index_tfidf.json", "w") as file: # storing for reusability
    json.dump(inverted_index, file)
inverted_index = json.load(open("/Users/saifdev/Desktop/ADMHW3/inverted_index_tfidf.json", "r")) # loading for use

#### 2.2.2) Execute the query
In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the tfIdf representations of the documents.

First, I will take the user input and normalize it.

In [25]:
user_input = input("Add your Input")
normalized_query = normalize_query(user_input)

Time to compute and get the TF-IDF scores for our query:

In [26]:
query_tfidf = calculate_query_tfidf(normalized_query, word_and_appearances, df)
query_tfidf

{'advanc': 0.84, 'knowledg': 0.57}

I will now compute the intersection list of query and documents.

In [27]:
appearances = list()
[appearances.append(word_and_appearances[word]) for word in normalized_query]
# initialising the set with the value of the first list of appearances in the list of appearances of the word
intersection_list = set(appearances[0])
for appearance in appearances[1:]: # for the rest of the terms in the list of term ids
    intersection_list.intersection_update(appearance)
intersection_list = sorted(list(intersection_list))

I will not compute the similarity scores using cosine similarity with respect to the tf-idf scores. I will store these scores in a dictionary named as similarity_scores where each key is the document_id (row of the dataframe) and value is the cosine similarity of the document and query

In [28]:
similarity_scores = {}
for document_i in intersection_list:
    document_vector = df.at[document_i, "description_clean"] # getting the document vector at the row of interest
    # calculated the tf_idf of the document vector. It will result in words as keys and values as tf-idf scores
    document_tfidf = calculate_document_tfidf(document_vector, word_and_appearances, df) 
    # Compute cosine similarity
    dot_product = 0
    for word in normalized_query:
        if word in document_tfidf:
# computing the dot product of only the query's tfidf score and the tfidf score of that word in the document
            dot_product += query_tfidf[word] * document_tfidf[word] 

    norm_doc_i = np.linalg.norm(list(document_tfidf.values())) # computing norm of the doc vector
    norm_query = np.linalg.norm(list(query_tfidf.values()) )# computing norm of the query vector
    
    if norm_doc_i != 0 and norm_query != 0: # only the non zero results
        cosine_similarity_doc_i_query = dot_product / (norm_doc_i * norm_query)
        similarity_scores[document_i] = cosine_similarity_doc_i_query

Using heap data structure to retrieve top 10 similarity scores:

In [29]:
top_10_documents = heapq.nlargest(10, similarity_scores, key=similarity_scores.get)

Finally, bringing the result of the query search.

In [30]:
results = return_results(top_10_documents, similarity_scores, df)
results

,courseName,universityName,description,url,Cosine_Similarity
0,Advanced Healthcare Practice - MSc,Cardiff University,Why study this courseOur MSc Advanced Healthca...,www.findamasters.com/masters-degrees/course/ad...,0.354660
1,Advanced Clinical Practice MSc,University of Greenwich,Learn essential strategies and prepare for lea...,www.findamasters.com/masters-degrees/course/ad...,0.353597
2,Advanced Computing MSc,King’s College London,Our Advanced Computing MSc provides knowledge ...,www.findamasters.com/masters-degrees/course/ad...,0.347863
3,Advancing Practice - MSc,University of Northampton,Our MSc Advancing Practice awards support the ...,www.findamasters.com/masters-degrees/course/ad...,0.337683
4,Advanced Clinical Practice - MSc,Canterbury Christ Church University,Gain the knowledge and skills needed to become...,www.findamasters.com/masters-degrees/course/ad...,0.291885
5,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,This course offers a broad range of advanced s...,www.findamasters.com/masters-degrees/course/ad...,0.290262
6,Advanced Professional Practice (MSc),University of Gloucestershire,Our lecturers are research active experts who ...,www.findamasters.com/masters-degrees/course/ad...,0.281497
7,Advanced Clinical Practice - MSc,University of Northampton,Our MSc Advanced Clinical Practice course aims...,www.findamasters.com/masters-degrees/course/ad...,0.280487
8,Advanced Biomedical Engineering - MSc,University of Bradford,Biomedical engineering is a fast evolving inte...,www.findamasters.com/masters-degrees/course/ad...,0.280119
9,Advanced Practice in Healthcare - MSc/PGDip/PG...,University of Birmingham,The MSc Advanced Practice in Healthcare is aim...,www.findamasters.com/masters-degrees/course/ad...,0.275154
